In [1]:
import numpy as np
from scipy import stats, optimize, integrate
from enum import Enum
from typing import List, Dict, Union, Optional
from dataclasses import dataclass

class ItemType(Enum):
    BINARY = "binary"
    LIKERT_5 = "likert_5"
    SCALE_10 = "scale_10"
    SINGLE_SELECT = "single_select"
    MULTI_SELECT = "multi_select"

@dataclass
class ItemParameters:
    discrimination: float  # a-parameter
    difficulty: float     # b-parameter
    guessing: float      # c-parameter (for binary/multiple choice)
    item_type: ItemType
    options: Optional[int] = None  # For multiple choice items

class Item:
    def __init__(self, item_id: str, params: ItemParameters):
        self.item_id = item_id
        self.params = params
    
    def response_probability(self, theta: float) -> Union[float, np.ndarray]:
        """Calculate the probability of each possible response given theta."""
        a, b, c = self.params.discrimination, self.params.difficulty, self.params.guessing
        
        if self.params.item_type == ItemType.BINARY:
            # 3PL model for binary items
            return c + (1 - c) / (1 + np.exp(-a * (theta - b)))
        
        elif self.params.item_type == ItemType.LIKERT_5:
            # Graded Response Model for Likert scale
            boundaries = np.array([b - 1, b - 0.5, b, b + 0.5, b + 1])
            probs = 1 / (1 + np.exp(-a * (theta - boundaries)))
            return np.diff(np.concatenate(([1], probs, [0])))
        
        elif self.params.item_type == ItemType.SCALE_10:
            # Generalized Partial Credit Model for scale
            steps = np.linspace(b - 2, b + 2, 9)  # 9 boundaries for 10 categories
            logits = a * (theta - steps)
            cumsum_logits = np.cumsum(logits)
            numerator = np.exp(cumsum_logits)
            denominator = 1 + np.sum(numerator)
            return np.diff(np.concatenate(([0], numerator / denominator, [1])))
        
        elif self.params.item_type in [ItemType.SINGLE_SELECT, ItemType.MULTI_SELECT]:
            # Modified 3PL for multiple choice
            n_options = self.params.options
            base_prob = c + (1 - c) / (1 + np.exp(-a * (theta - b)))
            if self.params.item_type == ItemType.SINGLE_SELECT:
                wrong_prob = (1 - base_prob) / (n_options - 1)
                probs = np.full(n_options, wrong_prob)
                probs[0] = base_prob  # Assuming first option is correct
                return probs
            else:
                return base_prob  # For multi-select, return probability for each option

class AdaptiveTester:
    def __init__(self, item_pool: List[Item], theta_prior_mean: float = 0, theta_prior_sd: float = 1):
        self.item_pool = item_pool
        self.administered_items = []
        self.responses = []
        self.theta_estimate = theta_prior_mean
        self.theta_sd = theta_prior_sd
        
    def select_next_item(self) -> Item:
        """Select next item based on maximum information at current theta estimate."""
        max_info = float('-inf')
        best_item = None
        
        for item in self.item_pool:
            if item not in self.administered_items:
                info = self._item_information(item, self.theta_estimate)
                if info > max_info:
                    max_info = info
                    best_item = item
        
        return best_item
    
    def _item_information(self, item: Item, theta: float) -> float:
        """Calculate Fisher information for an item at given theta."""
        h = 1e-5
        p = item.response_probability(theta)
        
        if isinstance(p, np.ndarray):
            # For polytomous items
            p_plus_h = item.response_probability(theta + h)
            p_minus_h = item.response_probability(theta - h)
            derivative = (p_plus_h - p_minus_h) / (2 * h)
            return np.sum(derivative ** 2 / (p * (1 - p)))
        else:
            # For binary items
            derivative = (item.response_probability(theta + h) - 
                        item.response_probability(theta - h)) / (2 * h)
            return derivative ** 2 / (p * (1 - p))
    
    def update_theta(self, item: Item, response: Union[int, List[int]]):
        """Update theta estimate using MLE or EAP."""
        self.administered_items.append(item)
        self.responses.append(response)
        
        # Using EAP estimation
        theta_range = np.linspace(self.theta_estimate - 4*self.theta_sd,
                                self.theta_estimate + 4*self.theta_sd, 100)
        
        likelihood = self._calculate_likelihood(theta_range)
        prior = stats.norm.pdf(theta_range, self.theta_estimate, self.theta_sd)
        posterior = likelihood * prior
        posterior /= integrate.simps(posterior, theta_range)
        
        # Update theta estimate and SD
        self.theta_estimate = integrate.simps(theta_range * posterior, theta_range)
        variance = integrate.simps((theta_range - self.theta_estimate)**2 * posterior, theta_range)
        self.theta_sd = np.sqrt(variance)
    
    def _calculate_likelihood(self, theta_range: np.ndarray) -> np.ndarray:
        """Calculate likelihood of all responses at given theta values."""
        likelihood = np.ones_like(theta_range)
        
        for item, response in zip(self.administered_items, self.responses):
            for theta_idx, theta in enumerate(theta_range):
                probs = item.response_probability(theta)
                
                if isinstance(response, list):  # Multi-select
                    for r in response:
                        likelihood[theta_idx] *= probs[r]
                else:  # Single response
                    if isinstance(probs, np.ndarray):
                        likelihood[theta_idx] *= probs[response]
                    else:
                        likelihood[theta_idx] *= probs if response == 1 else (1 - probs)
        
        return likelihood



In [2]:

# Create sample items
items = [
    Item("q1", ItemParameters(1.5, 0.0, 0.2, ItemType.BINARY)),
    Item("q2", ItemParameters(1.2, 0.5, 0.0, ItemType.LIKERT_5)),
    Item("q3", ItemParameters(1.0, -0.5, 0.0, ItemType.SCALE_10)),
    Item("q4", ItemParameters(1.3, 0.2, 0.25, ItemType.SINGLE_SELECT, options=4)),
    Item("q5", ItemParameters(1.1, 0.3, 0.2, ItemType.MULTI_SELECT, options=3))
]

# Initialize adaptive tester
tester = AdaptiveTester(items)

# Simulate a test session
for _ in range(3):
    next_item = tester.select_next_item()
    if next_item is None:
        break
        
    # Simulate response (in practice, this would come from the test taker)
    true_theta = 0.5  # True ability level
    probs = next_item.response_probability(true_theta)
    
    if isinstance(probs, np.ndarray):
        response = np.random.choice(len(probs), p=probs)
    else:
        response = 1 if np.random.random() < probs else 0
        
    tester.update_theta(next_item, response)
    print(f"Current theta estimate: {tester.theta_estimate:.3f} ± {tester.theta_sd:.3f}")

C:\Users\steph\AppData\Local\Temp\ipykernel_43936\2758907593.py:94: RuntimeWarning: divide by zero encountered in divide
  return np.sum(derivative ** 2 / (p * (1 - p)))


ValueError: probabilities are not non-negative